# Imports

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


# Part A

## Normalization of data

- We first read both files since the dataset was provided as train and test not as one csv file
- We then dropped the header row from both tables and concatenated them together to create the dataset
- We converted the data into a numpy array for faster operations
- We then seperated the labels into `y` and the features in `X` and set their data types
- Feature values were normalized to take value between `0 and 1`
- Afterwards we split the data into train, validation, and test

In [ ]:
data=pd.read_csv("dataset/mnist_test.csv")
data2=pd.read_csv("dataset/mnist_train.csv")
data2=data2.drop(data2.index[0])
data=data.drop(data.index[0])
data=pd.concat([data,data2])

data_np = data.to_numpy()

# Separate labels (first column) and features (remaining columns)
y = data_np[:, 0].astype(int)
X = data_np[:, 1:].astype(float)

# Normalize pixel values to [0, 1]
X = X / 255.0

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.4, stratify=y, random_state=42
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
)


  - `reshape()` is a numpy function that changes the dimension of a numpy array without changing the data
  - since the images were in the form of a flattened vector, we can change them back into 28x28 images using the `reshape() `
  - -1 -> it tells the function to automatically calculate this dimesion according to the other dimensions we will specifiy later (this one returns the number of images)
  - 1 -> number of color channels (only 1 since the images are grayscale)
  - 28 -> height 
  - 28 -> width

In [ ]:

# Reshape for neural networks
X_train_nn = X_train.reshape(-1, 1, 28, 28)
X_val_nn = X_val.reshape(-1, 1, 28, 28)
X_test_nn = X_test.reshape(-1, 1, 28, 28)


In [ ]:

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# For neural networks (keep image shape)
X_train_nn_tensor = torch.tensor(X_train_nn, dtype=torch.float32)
X_val_nn_tensor = torch.tensor(X_val_nn, dtype=torch.float32)
X_test_nn_tensor = torch.tensor(X_test_nn, dtype=torch.float32)

# Create TensorDatasets
train_dataset_flat = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset_flat = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset_flat = TensorDataset(X_test_tensor, y_test_tensor)

train_dataset_nn = TensorDataset(X_train_nn_tensor, y_train_tensor)
val_dataset_nn = TensorDataset(X_val_nn_tensor, y_val_tensor)
test_dataset_nn = TensorDataset(X_test_nn_tensor, y_test_tensor)

# Create DataLoaders
batch_size = 64

train_loader_flat = DataLoader(train_dataset_flat, batch_size=batch_size, shuffle=True)
val_loader_flat = DataLoader(val_dataset_flat, batch_size=batch_size, shuffle=False)
test_loader_flat = DataLoader(test_dataset_flat, batch_size=batch_size, shuffle=False)

train_loader_nn = DataLoader(train_dataset_nn, batch_size=batch_size, shuffle=True)
val_loader_nn = DataLoader(val_dataset_nn, batch_size=batch_size, shuffle=False)
test_loader_nn = DataLoader(test_dataset_nn, batch_size=batch_size, shuffle=False)

## Binary logistic regression 

In [ ]:
# Filter only 0 and 1 from training set
train_mask = (y_train == 0) | (y_train == 1)
val_mask = (y_val == 0) | (y_val == 1)
test_mask = (y_test == 0) | (y_test == 1)

X_train_bin = X_train_tensor[train_mask]
y_train_bin = y_train_tensor[train_mask]

X_val_bin = X_val_tensor[val_mask]
y_val_bin = y_val_tensor[val_mask]

X_test_bin = X_test_tensor[test_mask]
y_test_bin = y_test_tensor[test_mask]

# Create dataloaders
batch_size = 64
train_loader_bin = DataLoader(TensorDataset(X_train_bin, y_train_bin), batch_size=batch_size, shuffle=True)
val_loader_bin = DataLoader(TensorDataset(X_val_bin, y_val_bin), batch_size=batch_size, shuffle=False)
test_loader_bin = DataLoader(TensorDataset(X_test_bin, y_test_bin), batch_size=batch_size, shuffle=False)


In [ ]:
input_dim = 784  # flattened MNIST size

# Weights & bias
W = torch.zeros(input_dim, 1, dtype=torch.float32, requires_grad=True)
b = torch.zeros(1, dtype=torch.float32, requires_grad=True)

learning_rate = 0.01


In [ ]:
def sigmoid(z):
    return 1 / (1 + torch.exp(-z))

def binary_cross_entropy(pred, target):
    # Adding small epsilon for numerical stability
    eps = 1e-8
    return -(target * torch.log(pred + eps) + (1 - target) * torch.log(1 - pred + eps)).mean()


In [ ]:
def evaluate(loader):
    correct = 0
    total = 0
    loss_sum = 0
    with torch.no_grad():
        for X_batch, y_batch in loader:
            y_batch = y_batch.unsqueeze(1).float()
            logits = X_batch @ W + b
            y_pred = sigmoid(logits)
            loss = binary_cross_entropy(y_pred, y_batch)
            predicted = (y_pred >= 0.5).int()
            correct += (predicted.squeeze() == y_batch.squeeze()).sum().item()
            total += y_batch.size(0)
            loss_sum += loss.item()
    return loss_sum / len(loader), correct / total

In [ ]:
train_losses, val_losses = [], []
train_accs, val_accs = [], []

epochs = 20

for epoch in range(epochs):
    total_loss = 0
    correct = 0
    total = 0

    for X_batch, y_batch in train_loader_bin:
        y_batch = y_batch.unsqueeze(1).float()
        logits = X_batch @ W + b
        y_pred = sigmoid(logits)

        loss = binary_cross_entropy(y_pred, y_batch)
        loss.backward()

        with torch.no_grad():
            W -= learning_rate * W.grad
            b -= learning_rate * b.grad

        W.grad.zero_()
        b.grad.zero_()

        total_loss += loss.item()
        predicted = (y_pred >= 0.5).int()
        correct += (predicted.squeeze() == y_batch.squeeze()).sum().item()
        total += y_batch.size(0)

    train_loss = total_loss / len(train_loader_bin)
    train_acc = correct / total
    val_loss, val_acc = evaluate(val_loader_bin)

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs.append(train_acc)
    val_accs.append(val_acc)


In [ ]:
plt.figure(figsize=(12,5))

# Loss curves
plt.subplot(1,2,1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training & Validation Loss")
plt.legend()

# Accuracy curves
plt.subplot(1,2,2)
plt.plot(train_accs, label='Train Accuracy')
plt.plot(val_accs, label='Validation Accuracy')
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Training & Validation Accuracy")
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
all_preds = []
all_labels = []
with torch.no_grad():
    for X_batch, y_batch in test_loader_bin:
        logits = X_batch @ W + b
        y_pred = sigmoid(logits)
        preds = (y_pred >= 0.5).int().squeeze().numpy()
        all_preds.extend(preds)
        all_labels.extend(y_batch.numpy())

cm = confusion_matrix(all_labels, all_preds)

plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=[0,1], yticklabels=[0,1])
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix (Test Set)")
plt.show()

test_acc = np.mean(np.array(all_preds) == np.array(all_labels))
print(f"Final Test Accuracy: {test_acc:.4f}")



## Softmax Regression Implementation

In [ ]:
# Use the full dataset for multi-class classification (digits 0–9)
X_train_mc = torch.tensor(X_train, dtype=torch.float32)
y_train_mc = torch.tensor(y_train, dtype=torch.long)
X_val_mc = torch.tensor(X_val, dtype=torch.float32)
y_val_mc = torch.tensor(y_val, dtype=torch.long)
X_test_mc = torch.tensor(X_test, dtype=torch.float32)
y_test_mc = torch.tensor(y_test, dtype=torch.long)

num_features = X_train.shape[1]
num_classes = 10

W = torch.randn(num_features, num_classes, dtype=torch.float32, requires_grad=True)
b = torch.zeros(num_classes, dtype=torch.float32, requires_grad=True)


In [ ]:
def softmax(z):
    exp_z = torch.exp(z - z.max(dim=1, keepdim=True).values)  # numerical stability
    return exp_z / exp_z.sum(dim=1, keepdim=True)

def cross_entropy(pred, target):
    # target is a vector of class indices
    n = target.shape[0]
    eps = 1e-8
    log_likelihood = -torch.log(pred[range(n), target] + eps)
    return log_likelihood.mean()


In [ ]:
lr = 0.01
epochs = 50

train_losses_mc, val_losses_mc = [], []
train_accs_mc, val_accs_mc = [], []

for epoch in range(epochs):
    # Forward pass
    logits = X_train_mc @ W + b
    probs = softmax(logits)
    loss = cross_entropy(probs, y_train_mc)
    
    # Backprop
    loss.backward()
    
    # Gradient descent
    with torch.no_grad():
        W -= lr * W.grad
        b -= lr * b.grad
        W.grad.zero_()
        b.grad.zero_()
    
    # Training accuracy
    train_pred = probs.argmax(dim=1)
    train_acc = (train_pred == y_train_mc).float().mean().item()
    
    # Validation
    with torch.no_grad():
        val_logits = X_val_mc @ W + b
        val_probs = softmax(val_logits)
        val_loss = cross_entropy(val_probs, y_val_mc)
        val_pred = val_probs.argmax(dim=1)
        val_acc = (val_pred == y_val_mc).float().mean().item()
    
    train_losses_mc.append(loss.item())
    val_losses_mc.append(val_loss.item())
    train_accs_mc.append(train_acc)
    val_accs_mc.append(val_acc)
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}, Val Acc: {val_acc:.4f}")


In [ ]:
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(train_losses_mc, label="Train Loss")
plt.plot(val_losses_mc, label="Val Loss")
plt.legend(); plt.title("Softmax Regression Loss")

plt.subplot(1,2,2)
plt.plot(train_accs_mc, label="Train Accuracy")
plt.plot(val_accs_mc, label="Validation Accuracy")
plt.legend(); plt.title("Softmax Regression Accuracy")

plt.show()
with torch.no_grad():
    test_logits = X_test_mc @ W + b
    test_probs = softmax(test_logits)
    test_pred = test_probs.argmax(dim=1)
    test_acc = (test_pred == y_test_mc).float().mean().item()

print(f"Final Test Accuracy: {test_acc*100:.2f}%")

cm = confusion_matrix(y_test_mc, test_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap='Blues')
plt.title("Softmax Regression Confusion Matrix")
plt.show()

print("\nPer-Class Accuracy:")
print(classification_report(y_test_mc, test_pred))

# Part B

### Custom Neural Network Architecture

The **FullyConnectedNN** class implements the ANN. My implementation allows for different depths of the linear NN layers. Some important functions used:
- **nn.Linear**: instantiates a linear transformation layer, that applies the formula `y = xW^T + b`
- **nn.init.kaiming_normal_**: initialises the weights for the layers that use **ReLU** activation function and its variants to account for the fact that on average 50% of the weight would be 0, as **`RelU(x) = max(0, x)`**
- **nn.init.xavier_normal_**: initialises the weights for the layers that use activation functions that have zero mean and unit variance, such as the sigmoid function that is used in softmax regression


In [ ]:
class FullyConnectedNN(nn.Module):
    def __init__(self, input_size=784, hidden_sizes=[128, 64], num_classes=10):
        super(FullyConnectedNN, self).__init__()
    
        layers = []
        in_features = input_size
        
        for h in hidden_sizes:
            layers.append(nn.Linear(in_features, h))
            in_features = h
        
        layers.append(nn.Linear(in_features, num_classes))
        
        self.layers = nn.ModuleList(layers)

        for layer in self.layers[:-1]:
            nn.init.kaiming_normal_(layer.weight)
        nn.init.xavier_normal_(self.layers[-1].weight)

    def forward(self, x):
        if x.ndim == 4:
            x = x.view(x.size(0), -1)

        for layer in self.layers[:-1]:
            x = F.relu(layer(x))

        return self.layers[-1](x)
                

### Training Infrastructure

Auto-detection of a GPU and using it if exists. Otherwise, fall back onto CPU.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Training on: {device}")

The **accuracy** function calculates the accuracy per batch by:
- Taking the **outputs**, which are the raw logits, and the **labels**
- Getting the prediction by getting the maximum value using **argmax**
- Calculating the mean of how many predictions actually matched the labels

In [ ]:
def accuracy(outputs, labels):
    preds = outputs.argmax(dim=1)
    return (preds == labels).float().mean().item()

In [ ]:
def train_model(model, train_loader, val_loader, epochs=10, lr=0.01):
    model = model.to(device) # Move model to device
    
    criterion = nn.CrossEntropyLoss() # combines softmax and negative log likelihood
    
    # Stochastic Gradient Descent optimizer
    # model.parameters() gives all learnable parameters of the model to be optimized
    optimizer = optim.SGD(model.parameters(), lr=lr)
    
    train_losses, val_losses = [], []
    train_accs, val_accs = [], []
    
    for epoch in range(epochs):
        model.train() # set the model to training mode
        
        running_loss = 0.0
        running_acc = 0.0
        
        for X, y in train_loader: # train the model in batches
            X, y = X.to(device), y.to(device)
            
            outputs = model(X) # calls the forward function and returns raw logits
            loss = criterion(outputs, y) # computes how far the outputs are from the true labels
            
            optimizer.zero_grad() # reset gradients before backpropagation to prevent unwanted accumulation
            loss.backward() # compute gradients via backpropagation
            optimizer.step() # update model parameters using w = w - lr * gradient
            
            running_loss += loss.item()
            running_acc += accuracy(outputs, y)
        
        train_losses.append(running_loss / len(train_loader))
        train_accs.append(running_acc / len(train_loader))

        model.eval() # set the model to validation mode
        val_loss, val_acc = 0.0, 0.0

        with torch.no_grad(): # disable gradient calculation during validation
            for X, y in val_loader:
                X, y = X.to(device), y.to(device)
                
                outputs = model(X)
                loss = criterion(outputs, y)
                
                val_loss += loss.item()
                val_acc += accuracy(outputs, y)
        
        val_losses.append(val_loss / len(val_loader))
        val_accs.append(val_acc / len(val_loader))
        
        # Log progress
        print(f"Epoch [{epoch+1}/{epochs}] "
            f"Train Loss: {train_losses[-1]:.4f}, Train Acc: {train_accs[-1]*100:.2f}% "
            f"| Val Loss: {val_losses[-1]:.4f}, Val Acc: {val_accs[-1]*100:.2f}%")
    
    return train_losses, val_losses, train_accs, val_accs


- Run the fully connected NN model training

In [ ]:
fc_nn_model = FullyConnectedNN()
train_losses, val_losses, train_accs, val_accs = train_model(
    fc_nn_model,
    train_loader_nn,
    val_loader_nn,
    epochs=10,
    lr=0.01
)

# Part C

- Performance Visualization

- Hyperparameter Analysis

- Model Comparison

# Part D

### Convolutional Neural Networks

In [ ]:
class DynamicCNN(nn.Module):
    def __init__(self, conv_channels=[32, 64], fc_sizes=[128], num_classes=10):
        super(DynamicCNN, self).__init__()
        
        layers = []
        in_channels = 1
        
        for out_channels in conv_channels:
            layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
            layers.append(nn.ReLU())
            layers.append(nn.MaxPool2d(kernel_size=2))
            in_channels = out_channels
        
        self.conv_layers = nn.Sequential(*layers)

        pool_count = conv_channels.__len__()
        spatial_size = 28 // (2 ** pool_count)

        fc_layers = []
        in_features = conv_channels[-1] * spatial_size * spatial_size

        for h in fc_sizes:
            fc_layers.append(nn.Linear(in_features, h))
            fc_layers.append(nn.ReLU())
            in_features = h

        fc_layers.append(nn.Linear(in_features, num_classes))
        self.fc_layers = nn.Sequential(*fc_layers)

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layers(x)
        return x

- Run the convolutional NN model training

In [ ]:
cnn_model = DynamicCNN()
train_losses, val_losses, train_accs, val_accs = train_model(
    cnn_model,
    train_loader_nn,
    val_loader_nn,
    epochs=10,
    lr=0.01
)